# Azure AI Agents with Model Context Protocol (MCP) Support

このノートブックでは、Model Context Protocol (MCP) ツールを使用して、外部のMCPサーバーを活用し、機能を強化できるインテリジェントエージェントを作成する方法を示します。


## 必要なNuGetパッケージのインストール

まず、Azure AI Agentsを操作するための基本機能を提供するAzure AI Agents Persistentパッケージをインストールする必要があります。


## キーレス認証の利点

このノートブックでは、**キーレス認証**の利点を示します。キーレス認証には以下のようなメリットがあります：
- ✅ **APIキーの管理が不要** - Azureのアイデンティティベース認証を使用
- ✅ **セキュリティの向上** - コードや設定に秘密情報を保存しない
- ✅ **資格情報の自動ローテーション** - Azureが資格情報のライフサイクルを管理
- ✅ **ロールベースのアクセス** - Azure RBACを使用して細かい権限設定が可能

`DefaultAzureCredential` は、利用可能な最適な資格情報ソースを自動的に使用します：
1. マネージドアイデンティティ（Azure上で実行されている場合）
2. Azure CLI資格情報（開発中の場合）
3. Visual Studio資格情報
4. 環境変数（設定されている場合）


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

AzureサービスでDefaultAzureCredentialを使用して認証するためにAzure Identityパッケージをインストールします。


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## 必要な名前空間のインポート

Azure AI AgentsとAzure Identityに必要な名前空間をインポートします。


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Client の設定 (キーなし認証)

以下の設定変数を構成し、**キーなし認証**を使用して PersistentAgentsClient を作成します:
- **projectEndpoint**: Azure AI Foundry プロジェクトのエンドポイント
- **modelDeploymentName**: デプロイされた AI モデルの名前 (GPT-4.1 nano)
- **mcpServerUrl**: MCP サーバーの URL (Microsoft Learn API)
- **mcpServerLabel**: MCP サーバーを識別するためのラベル
- **DefaultAzureCredential**: マネージド ID、Azure CLI、その他の認証ソースを使用 (API キーは不要)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCPツール定義の作成

Microsoft Learn MCPサーバーに接続するMCPツール定義を作成します。これにより、エージェントがMicrosoft Learnのコンテンツやドキュメントにアクセスできるようになります。


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AIエージェントの作成

指定されたモデルとMCPツールを使用して、永続的なAIエージェントを作成します。このエージェントは以下の設定で構成されています：
- GPT-4.1 nanoモデル
- MCPツールを活用するための指示
- Microsoft Learn MCPサーバーへのアクセス


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## スレッドを作成してメッセージを送信

会話スレッドを作成し、Azure OpenAIとOpenAIの違いについて尋ねるユーザーメッセージを送信します。これにより、エージェントがMCPツールを使用して正確な情報を提供する能力をテストします。


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCPツールリソースの設定（キーなし）

MCPツールリソースを設定します。完全にキーなしのアプローチを採用する場合、MCPサーバーがAzureのアイデンティティベース認証をサポートしている場合は、カスタムヘッダーを削除することができます。以下の例では、必要に応じてヘッダーを追加する方法を示していますが、キーなしのシナリオではこれらは不要な場合があります。


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## エージェント実行の開始

ユーザーのメッセージを処理するための実行を作成し、開始します。エージェントは設定されたMCPツールとリソースを使用して応答を生成します。


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## 実行状況の監視とツール承認の処理（キー不要）

エージェントの実行状況を監視し、必要なツール承認を処理します。このループでは以下を行います：
1. 実行が完了するか、アクションが必要になるまで待機
2. 必要に応じてMCPツール呼び出しを自動承認
3. キー不要の認証の場合、MCPサーバーがAzure IDをサポートしている場合はヘッダーが不要な場合があります


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## 会話結果を表示

スレッド内のすべてのメッセージを取得して表示します。ユーザーの質問とエージェントの回答の両方を含みます。メッセージはタイムスタンプと役割の指標とともに時系列順で表示されます。


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤った解釈について、当方は責任を負いません。
